# 概要
* 複数インデックスのマージにいよいよトライする
* treeは自動で要約つけてくれるの？？めっちゃ便利やん
* 訳としては完璧やん...

# 参考
* そもそもtree構造が前提なのでtreeのindexを確認する
* https://gpt-index.readthedocs.io/en/latest/how_to/composability.html
* https://note.com/npaka/n/n5d4ff4380055

In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
#Jupyterでも実行できるのは知らんかった...
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

# PromptHelperとは
* トークンの制約に対応するための汎用プロンプトサポートツール

# LLM（Large Language Model）
* PromptHelperの引数の大部分はLLMに関連する
* つまり最初にLLMが何かを理解する必要がある
* (そもそも入力されたPromptをparseするのはLLMなのでPromptHelperがLLMを引数にとるのは当然である)
* 詳細はここを見よ！
* https://gpt-index.readthedocs.io/en/latest/reference/llm_predictor.html

# そもそも
* LLMPredictorとはLangchainのLLMChainのラッパーであり、LlamdaIndexと統合することができる
* LLMChain ↔ LLMPrecitor ↔ LlamdaIndex
* Our LLMPredictor is a wrapper around Langchain’s LLMChain that allows easy integration into LlamaIndex.
* llmを引数にとり、デフォルトではOpenAI’s text-davinci-003が指定される
* max_input_size: int = 3900, num_output: int = 256
* デフォルトでは入力は3900トークンで出力は256トークン
* つまり長文の回答はできない

# 引数のllm
* OpenAI以外にいも複数のLLMをサポートしている
* OpenAI, Cohere, Hugging Face, etc
* Llama_indexは自身でLLMはサポートしておらず、あくまでもlangchainを介して呼び出しを行う？

In [22]:
from gpt_index import LLMPredictor, GPTSimpleVectorIndex, SimpleDirectoryReader

In [23]:
# OpenAIのAPIのラッパー
from langchain.llms import OpenAI

# 利用例

In [24]:
from llama_index import (
    GPTSimpleVectorIndex, 
    GPTSimpleKeywordTableIndex, 
    GPTListIndex, 
    SimpleDirectoryReader,
    GPTTreeIndex
)

from llama_index import (
    GPTKeywordTableIndex, 
    SimpleDirectoryReader, 
    LLMPredictor,
    PromptHelper
)

from gpt_index.indices.knowledge_graph.base import GPTKnowledgeGraphIndex
from langchain import OpenAI

#documents = SimpleDirectoryReader('story/dai').load_data()
documents = SimpleDirectoryReader('story/dragonball').load_data()

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 1024
# set maximum chunk overlap
max_chunk_overlap = 0
# separatorが"/n/n"だとチャンクサイズの計算でエラーになるっぽい。"。"の場合は動く模様。
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, separator="。", chunk_size_limit=512)

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_output))

# build index
#d_index = GPTSimpleVectorIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)
index = GPTTreeIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
INFO:root:> Building index from nodes: 1 chunks
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Write a summary of the following. Try to use only the information provided. Try to include as many key details as possible.\\n\\n\\n\\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\n\\u5730\\u7403\\u306e\\u4eba\\u91cc\\u96e2\\u308c\\u305f\\u5c71\\u5965\\u306b\\u4f4f\\u3080\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u306f\\u3042\\u308b\\u65e5\\u3001\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u6765\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3046\\u3002\\u305d\\u3053\\u3067\\u30017\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u3001\\u3069\\u3093\\u306a\\u9858\\u3044\\u3067\\u3082\\u4e00

DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=30320 request_id=d4ce49732c23c432c0597064d2b8ee3a response_code=200
DEBUG:root:
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めると神龍（シェンロン）が現れるということを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、天下一武道会に出場し準優勝を果たす。3年後の天下一武道会では、鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。

ピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得てピッコロ大魔王に闘いを挑み勝利する。天下一武道会終了後、悟空は牛魔王の娘・チチと結婚し、マジュニアと決勝戦で激突し初の天下一武道会優勝を飾る。

サイヤ人編では、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は孫悟飯を助けるため、ベジータと決闘し何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。

フリーザ編では、重傷で入院中の悟空に代わり、悟飯、クリリン

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=23502 request_id=5975ba5addaa621e9e39199196f6516c response_code=200
DEBUG:root:
ドラゴンボールのストーリーは、悟空が伝説の戦士・超サイヤ人へと覚醒し、フリーザを倒した後、ポルンガによって地球に帰還した悟飯たちが復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させるところから始まる。その後、未来からやってきたブルマとベジータの息子であるトランクスが悟空に2体の人造人間が絶望の未来をもたらすことを告げる。悟空は心臓病によって命を落とすことになるが、修行を積んでセルゲームに臨む。セルゲームでは悟空が命を落とし、悟飯が超サイヤ人2へと覚醒してセルを撃破する。その後、セルゲームより約7年後、高校生になった悟飯が天下一武道会に出場することを知った悟空は、自らも出場するために占いババの力によって1日だけこの世に戻る。天下一武道会の最中、界王神から恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。悟空とベジータが界王神界で魔人ブウとの最終決戦に臨み、ドラゴンボールの協力を得て魔人ブウを撃破する。その後、悟空は孫のパンと共に天下一武道会に出場し、魔人ブウの生まれ変わりである少年・ウーブと出会い、修行に旅立ち、物語は幕を閉じる。
DEBUG:root:> 0/19, summary: 
孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴ...
DEBUG:root:> 10/19, summary: 
ドラゴンボールのストーリーは、悟空が伝説の戦士・超サイヤ人へと覚醒し、フリーザを倒した後、ポ...
INFO:root:> [build_index_from_documents] Total LLM token usage: 6152 t

In [25]:
index.save_to_disk("dragon_tree.json")

In [26]:
#dir(index)
index.docstore

DocumentStore(docs={'8618eeae-fc69-4287-b646-83b56c5ac565': IndexGraph(text=None, doc_id='8618eeae-fc69-4287-b646-83b56c5ac565', embedding=None, extra_info=None, all_nodes={0: Node(text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る', doc_id='4c80c6b4-948a-4b4b-9aaf-92cd40575f49', embedding=None, extra_info=None, index=0, child_indices=set(), ref_doc_id='e47c1be0-5cc0-42a8-8824-36a9cdaa4af7', node_info=None), 1: Node(text='人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する', doc_id='59c8297d-eda0-4faf-8e52-b22ae7559678', embedding=None, extra_info=None, index=1, child_indices=set(), ref_doc_id='e47c1be0-5cc0-42a8-8824-36a9cdaa4af7', node_info=None), 2: Node(text='\n\nその後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は

In [8]:
index.docstore.docs

{'6b8a9ddd-0258-46ed-a3bd-d7733538354e': IndexGraph(text=None, doc_id='6b8a9ddd-0258-46ed-a3bd-d7733538354e', embedding=None, extra_info=None, all_nodes={0: Node(text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る', doc_id='6d3d2eec-f7fa-45cb-8e9c-3856ee0265dc', embedding=None, extra_info=None, index=0, child_indices=set(), ref_doc_id='84acc9e3-d68e-4dcf-9e6d-6ac7912cf0a5', node_info=None), 1: Node(text='人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する', doc_id='64bed7ae-c6a0-4ff5-93c5-1dd875b87920', embedding=None, extra_info=None, index=1, child_indices=set(), ref_doc_id='84acc9e3-d68e-4dcf-9e6d-6ac7912cf0a5', node_info=None), 2: Node(text='\n\nその後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボール

In [10]:
index.docstore.docs['6b8a9ddd-0258-46ed-a3bd-d7733538354e'].all_nodes

{0: Node(text='孫悟空少年編\n地球の人里離れた山奥に住む尻尾の生えた少年・孫悟空はある日、西の都からやって来た少女・ブルマと出会う。そこで、7つ集めると神龍（シェンロン）が現れ、どんな願いでも一つだけ叶えてくれるというドラゴンボールの存在を、さらに育ての親である孫悟飯の形見として大切に持っていた球がその1つ「四星球（スーシンチュウ）」であることを知り、ブルマと共に残りのドラゴンボールを探す旅に出る', doc_id='6d3d2eec-f7fa-45cb-8e9c-3856ee0265dc', embedding=None, extra_info=None, index=0, child_indices=set(), ref_doc_id='84acc9e3-d68e-4dcf-9e6d-6ac7912cf0a5', node_info=None),
 1: Node(text='人さらいのウーロンや盗賊のヤムチャらを巻き込んだボール探しの末、世界征服を企むピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する', doc_id='64bed7ae-c6a0-4ff5-93c5-1dd875b87920', embedding=None, extra_info=None, index=1, child_indices=set(), ref_doc_id='84acc9e3-d68e-4dcf-9e6d-6ac7912cf0a5', node_info=None),
 2: Node(text='\n\nその後、悟空は旅の途中に知り合った武術の達人・亀仙人の下で、後に親友となるクリリンと共に8か月間にわたる修行を積み、その成果を確かめるために世界一の武術の達人を決める天下一武道会に出場し、変装して出場していた亀仙人に敗れるも準優勝を果たす。悟空は再び修行の旅へと出発し、ドラゴンボールの悪用を企むレッドリボン軍との闘いや、孫悟飯との再会などを経てさらに強さを増していく', doc_id='de5a39d0-5b76-4114-8c1c-ee74c5796f2c', embedding=None, extra_info=None, index=2, child_indices=set

In [11]:
index.docstore.docs['6b8a9ddd-0258-46ed-a3bd-d7733538354e'].all_nodes[19]

Node(text='\n孫悟空少年編では、尻尾の生えた少年・孫悟空が西の都からやってきた少女・ブルマと出会い、ドラゴンボールを7つ集めると神龍（シェンロン）が現れるということを知り、ブルマと共に残りのドラゴンボールを探す旅に出る。ピラフ一味にボールを奪われ神龍を呼び出されるが、ウーロンがとっさに言い放った下らない願いを叶えてもらうことで一味の野望を阻止する。悟空は武術の達人・亀仙人の下で8か月間にわたる修行を積み、天下一武道会に出場し準優勝を果たす。3年後の天下一武道会では、鶴仙流の天津飯（てんしんはん）と闘うが、あと一歩のところで敗れ、前回と同じく準優勝に終わる。\n\nピッコロ大魔王編では、ピラフ一味によって復活したピッコロ大魔王によって、クリリンや亀仙人など悟空の仲間たちや多くの武道家たちが殺される。悟空は仇を討つため、ヤジロベーや仙猫カリンの協力を得てピッコロ大魔王に闘いを挑み勝利する。天下一武道会終了後、悟空は牛魔王の娘・チチと結婚し、マジュニアと決勝戦で激突し初の天下一武道会優勝を飾る。\n\nサイヤ人編では、息子の孫悟飯を儲けて平和な日々を過ごしていた悟空のもとに、実兄・ラディッツが宇宙より来襲し、自分が惑星ベジータの戦闘民族・サイヤ人であることを知らされる。悟空は孫悟飯を助けるため、ベジータと決闘し何とか辛勝し撤退させるが、多くの仲間を失うとともに、ピッコロの戦死により彼と一心同体であった神も死亡し、地球のドラゴンボールも消滅する。\n\nフリーザ編では、重傷で入院中の悟空に代わり、悟飯、クリリン、ブルマの3人が神とピッコロの故郷であるナメック星へ向かう。そこには地球で闘ったベジータや、界王すら畏怖する宇宙の帝王・フリーザとその', doc_id='e97383ce-87fc-4b96-b0b5-e997b79fc256', embedding=None, extra_info=None, index=19, child_indices={0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, ref_doc_id=None, node_info=None)

In [12]:
index.docstore.docs['6b8a9ddd-0258-46ed-a3bd-d7733538354e'].all_nodes[20]

Node(text='\nドラゴンボールのストーリーは、悟空が伝説の戦士・超サイヤ人へと覚醒し、フリーザを倒した後、悟飯たちが復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させるところから始まる。その後、未来から現れた人造人間・セルが地球の命運を賭けた武道会「セルゲーム」を開催し、悟空はセルとの闘いの中で命を落とす。その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒し、セルを撃破する。7年後、高校生になった悟飯が天下一武道会に出場することを知った悟空は、自らも出場するために占いババの力によって1日だけこの世に戻る。その最中、界王神から恐ろしい力を持つ魔人ブウの封印が解かれようとしていることを知らされる。魔人ブウは様々な人間との出会いからより邪悪で強力な魔人へと変貌し、悟空があの世に帰ったため、地球の命運は悟空の次男・孫悟天と少年トランクスの幼い二人に託される。悟空とベジータが界王神界で真の姿となった魔人ブウとの最終決戦に臨み、ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。10年後、悟空は孫のパンと共に天下一武道会に出場し、魔人ブウの生まれ変わりである少年・ウーブと出会い、物語は幕を閉じる。', doc_id='5bc546b4-6628-4e98-b68c-6b9b5d0f0e4f', embedding=None, extra_info=None, index=20, child_indices={10, 11, 12, 13, 14, 15, 16, 17, 18}, ref_doc_id=None, node_info=None)

In [13]:
index.query("魔人ブウの生まれ変わりは誰？")

INFO:root:> Starting query: 魔人ブウの生まれ変わりは誰？
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 2),where each item in the list corresponds to a summary.\\n---------------------\\n(1)  \\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u304c\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3044\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u30927\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u308b\\u3068\\u3044\\u3046\\u3053\\u3068\\u3092\\u77e5\\u308a\\u3001\\u30d6\\u30eb\\u30de\\u3068\\u5171\\u306b\\u6b8b\\u308a\\u306e\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 541
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2323 request_id=7ef655fbc13979f375720ec70f7b1f88 response_code=200
DEBUG:root:
ANSWER: 8
This summary was selected because it mentions that 10 years after the defeat of Majin Buu, Goku meets a young boy who is the reincarnation of Majin Buu. This is the most relevant to the question as it provides the answer to who the reincarnation of Majin Buu is.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 9),where each item in the list corresponds to a summary.
---------------------
(1) 怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザに勝利する。ポルンガによって地球に帰還した悟飯たちは復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させた。一方の悟空も爆発するナメック星を辛くも脱出、ヤードラット星に漂着し一命を取り留めた

(2)   人造人間・セル編 ナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球を襲撃するが、謎の超サイヤ人によって撃退される。トランクスと名乗るその青年は、自分は未来

Response(response='魔人ブウの生まれ変わりは、少年のウーブです。', source_nodes=[SourceNode(source_text='危機に陥った悟飯らを救うため現世に舞い戻った悟空とベジータは、界王神界で真の姿となった魔人ブウとの最終決戦に臨む。ドラゴンボールの協力もあり、地球・ナメック星・あの世の人々のエネルギーによって作り上げられた超特大の元気玉によって魔人ブウは完全に消滅する。\n\nそれから10年後、悟空は孫のパンと共に天下一武道会に久しぶりに出場し、魔人ブウの生まれ変わりである少年・ウーブと出会う', doc_id='84acc9e3-d68e-4dcf-9e6d-6ac7912cf0a5', extra_info=None, node_info=None, similarity=None)], extra_info=None)

In [14]:
index.query("セルを倒したのは誰？")

INFO:root:> Starting query: セルを倒したのは誰？
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Some choices are given below. It is provided in a numbered list (1 to 2),where each item in the list corresponds to a summary.\\n---------------------\\n(1)  \\u5b6b\\u609f\\u7a7a\\u5c11\\u5e74\\u7de8\\u3067\\u306f\\u3001\\u5c3b\\u5c3e\\u306e\\u751f\\u3048\\u305f\\u5c11\\u5e74\\u30fb\\u5b6b\\u609f\\u7a7a\\u304c\\u897f\\u306e\\u90fd\\u304b\\u3089\\u3084\\u3063\\u3066\\u304d\\u305f\\u5c11\\u5973\\u30fb\\u30d6\\u30eb\\u30de\\u3068\\u51fa\\u4f1a\\u3044\\u3001\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\u30eb\\u30927\\u3064\\u96c6\\u3081\\u308b\\u3068\\u795e\\u9f8d\\uff08\\u30b7\\u30a7\\u30f3\\u30ed\\u30f3\\uff09\\u304c\\u73fe\\u308c\\u308b\\u3068\\u3044\\u3046\\u3053\\u3068\\u3092\\u77e5\\u308a\\u3001\\u30d6\\u30eb\\u30de\\u3068\\u5171\\u306b\\u6b8b\\u308a\\u306e\\u30c9\\u30e9\\u30b4\\u30f3\\u30dc\\u30fc\\

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 493
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2194 request_id=ab1d7fd1157d7910a2d794cd12d7a006 response_code=200
DEBUG:root:
ANSWER: 5
This summary was selected because it explains how Son Gohan, with the help of the spirit of his father, Goku, was able to defeat Cell. It also explains how the Dragon Balls were used to revive those killed by Cell.
DEBUG:root:>[Level 1] current prompt template: Some choices are given below. It is provided in a numbered list (1 to 9),where each item in the list corresponds to a summary.
---------------------
(1) 怒りを爆発させた悟空は伝説の戦士・超（スーパー）サイヤ人へと覚醒。フルパワーを解放したフリーザに勝利する。ポルンガによって地球に帰還した悟飯たちは復活したドラゴンボールによりサイヤ人やフリーザ一味に殺された人々を蘇生させた。一方の悟空も爆発するナメック星を辛くも脱出、ヤードラット星に漂着し一命を取り留めた

(2)   人造人間・セル編 ナメック星での闘いから約1年後、密かに生き延びていたフリーザとその一味が地球を襲撃するが、謎の超サイヤ人によって撃退される。トランクスと名乗るその青年は、自分は未来からやってきたブルマとベジータの息子であることを悟空にだけ明かすと同時に、悟空は心臓病によって命

Response(response='悟飯がセルを倒した。', source_nodes=[SourceNode(source_text='だが、その遺志を受け継いだ息子・悟飯が超サイヤ人2へと覚醒、父・悟空の幻影と共にかめはめ波を放ちセルを撃破。セルゲームを制した悟飯たちは、ドラゴンボールによりセルに殺された人々を蘇生させるが、悟空は自分が悪人を引き寄せているという理由で生き返りを拒否し、あの世に残ることを選ぶ', doc_id='84acc9e3-d68e-4dcf-9e6d-6ac7912cf0a5', extra_info=None, node_info=None, similarity=None)], extra_info=None)

In [ ]:
index.query("内容を詳しくまとめてください", mode="summarize")

In [16]:
documents = SimpleDirectoryReader('story/dai').load_data()
index = GPTTreeIndex(documents, prompt_helper=prompt_helper, llm_predictor=llm_predictor)

DEBUG:root:> [SimpleDirectoryReader] Total files added: 1
INFO:root:> Building index from nodes: 1 chunks
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["Write a summary of the following. Try to use only the information provided. Try to include as many key details as possible.\\n\\n\\n\\u5e8f\\u7ae0\\uff08\\u30c7\\u30eb\\u30d1! \\u30a4\\u30eb\\u30a4\\u30eb! / \\u30c0\\u30a4\\u7206\\u767a!!!\\uff09\\n\\u9b54\\u738b\\u30cf\\u30c9\\u30e9\\u30fc\\u7387\\u3044\\u308b\\u9b54\\u738b\\u8ecd\\u3068\\u52c7\\u8005\\u7387\\u3044\\u308b\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u6226\\u3044\\u306f\\u3001\\u52c7\\u8005\\u30d1\\u30fc\\u30c6\\u30a3\\u306e\\u52dd\\u5229\\u3067\\u7d42\\u308f\\u308a\\u3001\\u9b54\\u738b\\u306e\\u914d\\u4e0b\\u3067\\u3042\\u3063\\u305f\\u30e2\\u30f3\\u30b9\\u30bf\\u30fc\\u3082\\u9b54\\u738b\\u306e\\u652f\\u914d\\u304b\\u3089\\u89e3\\u304d\\u653e\\u305f\\u308c\\u300

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (2): api.openai.com:443
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=25589 request_id=00ca733695647418e93a55c9d3dee5e6 response_code=200
DEBUG:root:
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配から解き放たれ、世界に平和が訪れた。その後、モンスターの島・デルムリン島に漂着した少年ダイは、勇者になることを夢見ていた。そんな中、謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。ダイはアバンの弟子となり、魔王を倒し平和を取り戻すべく、短期間のうちに秘められた力を開花させていく。15年ぶりに復活したハドラーが襲来し、ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。ダイ一行はロモス王国とパプニカ王国を占拠した魔王軍幹部の獣王クロコダインと魔剣戦士ヒュンケル、氷炎将軍フレイザードを撃破して奪還し、レオナの救出にも成功した。そして、大魔王バーンの力で蘇ったハドラーを倒すべく、ダイ一行は魔界へと向かう。ダイは次第に自分自身の力が人間として超絶したものであることによる他者からの視線に苦悩するようになり、竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。ダイ一行は魔王軍の本拠地である鬼岩城を破壊し、バーンパレスへと向かうが、魔宮の門によって閉ざされていた。そこで、バーンの真の目的を知ったバランがダイ一行と共闘することになる。
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai

DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=30869 request_id=c8244976f3e2ff366687f1ba1a055288 response_code=200
DEBUG:root:
ダイとバランは魔宮の門を破壊した後、ハドラーと対峙するが、バランはハドラーの体内に魔界の超強力爆弾「黒の核晶（くろのコア）」が埋め込まれていることを発見する。バランは黒の核晶が起動する前にハドラーから引き抜こうとするが、ミストバーンによって核晶は起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながらも命を落とす。ダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北するが、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、神の涙によって爆発は阻止される。ダイは竜の魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。最終的にダイの勝利で終わる。地上に帰還したダイが仲間たちに迎えられるが、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンをも倒したダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、ダイはポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもなお、ダイの姿は見つからなかったが、魔法宝
DEBUG:root:> 0/17, summary: 
十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配から解き放たれ、世界に平和...
DEBUG:root:> 10/17, summary: 
ダイとバランは魔宮の門を破壊した後、ハドラーと対峙するが、バランはハドラーの体内に魔界の超強...

In [17]:
index.save_to_disk("dai_tree.json")

In [20]:
index.docstore.docs['a38c38b8-2d56-4bcf-97c4-40a30a5edcb8'].all_nodes[17]

Node(text='\n十数年前、魔王ハドラーを倒した勇者パーティの勝利により、魔王の支配から解き放たれ、世界に平和が訪れた。その後、モンスターの島・デルムリン島に漂着した少年ダイは、勇者になることを夢見ていた。そんな中、謎の人物アバンとその弟子の魔法使いポップが現れ、島を破邪呪文・マホカトールで覆い、モンスターたちを魔王の支配から救った。ダイはアバンの弟子となり、魔王を倒し平和を取り戻すべく、短期間のうちに秘められた力を開花させていく。15年ぶりに復活したハドラーが襲来し、ダイは額に浮き出た謎の紋章の力でパワーアップし、ハドラーを退ける。ダイ一行はロモス王国とパプニカ王国を占拠した魔王軍幹部の獣王クロコダインと魔剣戦士ヒュンケル、氷炎将軍フレイザードを撃破して奪還し、レオナの救出にも成功した。そして、大魔王バーンの力で蘇ったハドラーを倒すべく、ダイ一行は魔界へと向かう。ダイは次第に自分自身の力が人間として超絶したものであることによる他者からの視線に苦悩するようになり、竜騎将バランが現れ、ダイの正体が伝説の存在「竜（ドラゴン）の騎士」であることを告げる。ダイ一行は魔王軍の本拠地である鬼岩城を破壊し、バーンパレスへと向かうが、魔宮の門によって閉ざされていた。そこで、バーンの真の目的を知ったバランがダイ一行と共闘することになる。', doc_id='73788984-eeb3-4285-a518-c5f56a169354', embedding=None, extra_info=None, index=17, child_indices={0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, ref_doc_id=None, node_info=None)

In [21]:
index.docstore.docs['a38c38b8-2d56-4bcf-97c4-40a30a5edcb8'].all_nodes[18]

Node(text='\nダイとバランは魔宮の門を破壊した後、ハドラーと対峙するが、バランはハドラーの体内に魔界の超強力爆弾「黒の核晶（くろのコア）」が埋め込まれていることを発見する。バランは黒の核晶が起動する前にハドラーから引き抜こうとするが、ミストバーンによって核晶は起動してしまい、バランはダイたちを守るために全ての力を使って爆発を押さえこみながらも命を落とす。ダイ一行はバーンとの直接対決において圧倒的な力の差の前に敗北するが、ハドラーがバーンに反逆し、攻撃を仕掛けた隙に脱出する。バーンは事前の爆撃で世界各地に投下していた複数の黒の核晶を使い、「地上界を消滅させ、地下にある魔界に太陽の光を降り注がせる」という真の目的を成し遂げようとするが、神の涙によって爆発は阻止される。ダイは竜の魔人となり、バーンも自らの魔力の源・鬼眼を解放し、鬼眼王と化す。最終的にダイの勝利で終わる。地上に帰還したダイが仲間たちに迎えられるが、キルバーンが現れ、自らの正体を明かす。キルバーンはバーンをも倒したダイ一行を全滅させるべく、隠していた黒の核晶を起動させて絶望する一同を嘲笑い、魔界に逃げ帰ろうとする。ダイとポップは核晶を空高く運んでいくが、ダイはポップを地上に戻し、一人爆発の閃光の中に消えた。数週間の捜索を経てもなお、ダイの姿は見つからなかったが、魔法宝', doc_id='42041fd2-8ae5-4968-902b-622c0594ca0a', embedding=None, extra_info=None, index=18, child_indices={10, 11, 12, 13, 14, 15, 16}, ref_doc_id=None, node_info=None)